# CCP over SocketCAN

> flashing interface for CCP over SocketCAN
> For now, only support CCP on CAN via VCAN or Kvaser CAN interface
> caution: requiring can-utils cansend

In [ ]:
#| default_exp ccp.socketcan

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from fastcore.test import *

In [ ]:
#| hide
from IPython.core.interactiveshell import InteractiveShell 

In [ ]:
#| export
import os
import sys
import git
import argparse
from InquirerPy import inquirer
from InquirerPy.validator import EmptyInputValidator
from InquirerPy.base.control import Choice
from pydantic import BaseModel, Field,  ConfigDict, model_validator, conlist, conint, computed_field, confloat, ValidationError
from pydantic.functional_validators import AfterValidator
from typing import Optional, TypedDict, Union, Literal
from typing_extensions import Annotated
from enum import StrEnum
from pprint import pprint, PrettyPrinter

In [ ]:
#| export
import subprocess
from multiprocessing import Manager
from multiprocessing.managers import DictProxy
import can
from cantools.database import Message as MessagerTpl
from cantools.database.can.database import Database
import contextlib

In [ ]:
#| export
import pandas as pd
import numpy as np
import struct

In [ ]:
# #|export
# from candycan.data_link_socketcan import done, send_msg

In [ ]:
#| export 
from candycan.a2l import (
    list_of_strings,
    XCPCalib,
    XCPData,
    XCPConfig,
    Get_XCPCalib_From_XCPJSon,
    Generate_Init_XCPData_From_A2L,
)


'/home/veos/devel/candycan'


In [ ]:
#| export
# from scapy.all import (
#     raw, rdpcap, wrpcap, load_contrib, hexdump,
#     ls, conf, load_layer, IP, Ether, TCP
# )
# Ether, TCP, hexdump, raw, rdpcap, load_contrib, conf, load_layer, 
# CANSocket, CAN, wrpcap, CCP, CRO, CONNECT, GET_SEED, UNLOCK, GET_DAQ_SIZE
from candycan.data_link.socketcan import (
    send_msg, done, signal_usr1, receive_message
)

'/home/veos/devel/candycan'


In [ ]:
#| hide
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#| export
pp = PrettyPrinter(indent=4, width=80, compact=True)

In [ ]:
#| export 
repo = git.Repo("./", search_parent_directories=True)  # get the Repo object of tspace
if os.path.basename(repo.working_dir) != "candycan":  # I'm in the parent repo!
    repo = repo.submodule("candycan").module()
pprint(repo.working_dir)

'/home/veos/devel/candycan'


In [ ]:
#| export
def get_argparser() -> argparse.ArgumentParser:
	"""Summary
	Get argument parser for command line arguments

	Returns:
		argparse.ArgumentParser: _description_
	"""
	parser = argparse.ArgumentParser(description='XCP Processing')

	parser.add_argument(
		'--protocol',
		type=str,
		choices=['CCP', 'XCP'],
		default='CCP',
		help='Protocol to use: CCP/XCP',
	)

	parser.add_argument(
		'--bus_type',
		type=str,
		choices=['SOCKET', 'VIRTUAL'],
		default='SOCKET',
		help='Bus type to use: SOCKET/VIRTUAL',
	)

	parser.add_argument(
		'--download',
		default=False,
		help='Download or upload: default is download(host->target)',
		action='store_true',
	)

	parser.add_argument(
		'--diff_mode',
		default=True,
		help='use differential mode for flashing',
		action='store_true',
	)

	parser.add_argument(
		'--diff_threshold',
		type=float,
		default=0.001,
		help='Threshold as different float value for differential mode',
	)

	parser.add_argument(
		'--bit_rate',
		type=int,
		choices=range(1_000_000),
		default=500_000,
		help='Bit rate for CAN bus, maximal 1Mbps',
	)

	parser.add_argument(
		'--time_out',
		type=float,
		default=1.0,
		help='Time out for CAN bus response',
	)

	parser.add_argument(
		"--station_address",
		type=int,
		default=0,  # 0 for all except for MOVE, which should be 1
		help='Station address of ECU for CCP protocol, MTA number 0/1',
	)

	parser.add_argument(
		"--download_can_id",
		type=int,
		default=630,
		help='Download CAN message ID for CCP protocol',
	)

	parser.add_argument(
		"--upload_can_id",
		type=int,
		default=631,
		help='Upload CAN message ID for CCP protocol',
	)

	parser.add_argument(
		'--a2l', 
		type=str,
        default=repo.working_dir+'/res/VBU_AI.json',
		help='a2l json file path')

	parser.add_argument(
		"--node-path",
		type=str,
		default=r"/PROJECT/MODULE[]",
		help="node path to search for calibration parameters",
	)

	parser.add_argument(
		"--leaves",
		type=list_of_strings,
		default=r"TQD_trqTrqSetNormal_MAP_v, " 
				r"VBU_L045A_CWP_05_09T_AImode_CM_single, " 
				r"Lookup2D_FLOAT32_IEEE, " 
				r"Lookup2D_X_FLOAT32_IEEE, " 
				r"Scalar_FLOAT32_IEEE, " 
				r"TQD_vVehSpd, "
				r"TQD_vSgndSpd_MAP_y, "
				r"TQD_pctAccPedPosFlt, "
				r"TQD_pctAccPdl_MAP_x",
			help="leaf nodes to search for",
	)

	parser.add_argument(
		'--channel_serial_number', 
		type=int,
		default=3,
		help='CAN channel serial number for flashing')

	parser.add_argument(
		'--input', 
		type=str,
        default=repo.working_dir+'/res/download.json',
		help='Input file path')

	parser.add_argument(
		'--output', 
		type=str, 
        default=repo.working_dir+'/res/output.json',
		help='Output file path')
	return parser

In [ ]:
show_doc(get_argparser)

---

### get_argparser

>      get_argparser ()

Summary
Get argument parser for command line arguments

Returns:
        argparse.ArgumentParser: _description_

In [ ]:
parser = get_argparser()
args = parser.parse_args(
    args=[
        '--protocol', 'CCP',
        '--bus_type', 'SOCKET', # 'VIRTUAL',  #
        '--download',
        '--diff_mode',
        '--diff_threshold', '0.001',
        '--bit_rate', '500_000',
        '--time_out', '1.0',
        '--station_address', 0,
        '--download_can_id', '630',
        '--upload_can_id', '631',
        '--a2l', repo.working_dir+'/res/VBU_AI.json',
        '--node-path', r'/PROJECT/MODULE[]',
        '--leaves', r'TQD_trqTrqSetNormal_MAP_v, VBU_L045A_CWP_05_09T_AImode_CM_single, Lookup2D_FLOAT32_IEEE, Lookup2D_X_FLOAT32_IEEE, Scalar_FLOAT32_IEEE, TQD_vVehSpd, TQD_vSgndSpd_MAP_y, TQD_pctAccPedPosFlt, TQD_pctAccPdl_MAP_x',
        '--channel_serial_number', '3',
        '--input', repo.working_dir+'/res/download.json',
        '--output', repo.working_dir+'/res/output.json',
    ]
)

In [ ]:
args.download_can_id, args.channel_serial_number, args.upload_can_id
args.diff_threshold, hex(args.station_address), args.time_out
args.__dict__

(630, 3, 631)

(0.001, '0x0', 1.0)

{'protocol': 'CCP',
 'bus_type': 'SOCKET',
 'download': True,
 'diff_mode': True,
 'diff_threshold': 0.001,
 'bit_rate': 500000,
 'time_out': 1.0,
 'station_address': 0,
 'download_can_id': 630,
 'upload_can_id': 631,
 'a2l': '/home/veos/devel/candycan/res/VBU_AI.json',
 'node_path': '/PROJECT/MODULE[]',
 'leaves': ['TQD_trqTrqSetNormal_MAP_v',
  'VBU_L045A_CWP_05_09T_AImode_CM_single',
  'Lookup2D_FLOAT32_IEEE',
  'Lookup2D_X_FLOAT32_IEEE',
  'Scalar_FLOAT32_IEEE',
  'TQD_vVehSpd',
  'TQD_vSgndSpd_MAP_y',
  'TQD_pctAccPedPosFlt',
  'TQD_pctAccPdl_MAP_x'],
 'channel_serial_number': 3,
 'input': '/home/veos/devel/candycan/res/download.json',
 'output': '/home/veos/devel/candycan/res/output.json'}

## Types definition

In [ ]:
#| export
BUS_TYPES = set(['SOCKET', 'VIRTUAL'])
# class BusType(StrEnum):
#     SOCKET = "SOCKET"
#     VIRTUAL = "VIRTUAL"

In [ ]:
#| export
def check_bus_type(b: str) -> str:
    """Summary
    Check if the CAN bus type is valid

    Args:
        b (str): Python CAN bus type to be checked

    Returns:
        str: Python CAN bus type if valid

    Raises:
        ValueError: if CAN bus type is invalid
    """
    if b.upper() not in BUS_TYPES:
        raise ValueError(f"Invalid Python CAN bus type: {b}, valid types are: {BUS_TYPES}")
    return b

BusType = Annotated[str, AfterValidator(check_bus_type)]

In [ ]:
#|export
class CANFilter(BaseModel):
    """Summary
    CAN filter for Python CAN bus

    Attributes:
        can_id (int): CAN message ID
        can_mask (int): CAN message mask
    """
    can_id: int = Field(default=630,gt=0,title="CAN message ID",description="CAN message ID")
    can_mask: int = Field(default=0x7ff,gt=0,title="CAN message mask",description="CAN message mask")

In [ ]:
#| export
class ScapyCANSpecs(BaseModel):
    bus_type: BusType = Field(frozen=True, default='VIRTUAL', description='Python CAN bus type')
    channel_serial_number: int = Field(frozen=True, default=3, ge=0, lt=500,description='CAN channel')
    download_can_id: int = Field(default=630, gt=0, description='Download CAN message ID')
    upload_can_id: int = Field(default=630, gt=0, description='Upload CAN message ID')
    can_filters: Optional[list[CANFilter]] = Field(default=None, description='CAN filters')
    bit_rate: int = Field(default=500_000, gt=0, lt=1_000_000, description='CAN bit rate')
    time_out: float = Field(default=1.0, gt=0.0, lt=10.0, description='CAN time out')
    station_address: int = Field(default=0x00, ge=0, lt=0xff, description='CAN station address')
    cntr: int = Field(default=0, ge=0, lt=1_000_000, description='CAN counter')
    receive_own_messages: bool = Field(default=True, description='Receive own messages')
    download_upload: bool = Field(default=True, description='Download if True or upload if False')
    diff_mode: bool = Field(default=False, description='Differential flashing (download)')
    diff_threshold: float = Field(default=1e-3, description='Threshold for float value difference')
    last_download_data: Optional[XCPData] = Field(default=None, description='Last download data')
    
    @computed_field
    def channel(self) -> Union[str, int]:
        """Summary
        Get the CAN channel as str for SocketCAN or virtual CAN, as int for Vector

        Returns:
            str: CAN channel for SocketCAN virtual CAN
            int: for Vector CAN
        """
        match self.bus_type:
            case 'VIRTUAL':
                return 'vcan' + str(self.channel_serial_number)
            case 'SOCKET':
                return 'can' + str(self.channel_serial_number)
            case 'VECTOR' | "KVASER":
                return self.channel_serial_number
            case 'PCAN':
                return 'PCAN_USBBUS' + str(self.channel_serial_number)
            case _:
                raise NotImplementedError(f"Bus type: {self.bus_type} not implemented yet")
                


In [ ]:
try:
    m = ScapyCANSpecs(can_type='NATIVE', bus_type='VIRTUAL')
except ValidationError as exc:
    print(exc)

pprint(m.model_dump())

{'bit_rate': 500000,
 'bus_type': 'VIRTUAL',
 'can_filters': None,
 'channel': 'vcan3',
 'channel_serial_number': 3,
 'cntr': 0,
 'diff_mode': False,
 'diff_threshold': 0.001,
 'download_can_id': 630,
 'download_upload': True,
 'last_download_data': None,
 'receive_own_messages': True,
 'station_address': 0,
 'time_out': 1.0,
 'upload_can_id': 630}


In [ ]:

def emulate_call(m: ScapyCANSpecs):
    m.cntr += 1

for i in range(3):
    emulate_call(m)
    print(f"{i}: counter {m.cntr}")

0: counter 1
1: counter 2
2: counter 3


## CCP code

In [ ]:
#| export
class CCPCommand(BaseModel):
    connect: int = Field(default=0x01, frozen=True, description='CCP connect command')
    set_mta: int = Field(default=0x02, frozen=True, description='CCP set memory transfer address command')
    disconnect: int = Field(default=0x07, frozen=True, description='CCP disconnect command')
    download: int = Field(default=0x03, frozen=True, description='CCP download command DNLOAD')
    download6: int = Field(default=0x23, frozen=True, description='CCP download command DNLOAD6')
    upload: int = Field(default=0x04, frozen=True, description='CCP upload command UPLOAD')
    short_upload: int = Field(default=0x0F, frozen=True, description='CCP short upload command SHORT_UP')
    get_seed: int = Field(default=0x12, frozen=True, description='CCP get seed command GET_SEED')
    get_ccp_version:  int = Field(default=0x1B, frozen=True, description='CCP get CCP version command GET_CCP_VERSION')
    
        # 0x01: "CONNECT",
        # 0x1B: "GET_CCP_VERSION",
        # 0x17: "EXCHANGE_ID",
        # 0x12: "GET_SEED",
        # 0x13: "UNLOCK",
        # 0x02: "SET_MTA",
        # 0x03: "DNLOAD",
        # 0x23: "DNLOAD_6",
        # 0x04: "UPLOAD",
        # 0x0F: "SHORT_UP",
        # 0x11: "SELECT_CAL_PAGE",
        # 0x14: "GET_DAQ_SIZE",
        # 0x15: "SET_DAQ_PTR",
        # 0x16: "WRITE_DAQ",
        # 0x06: "START_STOP",
        # 0x07: "DISCONNECT",
        # 0x0C: "SET_S_STATUS",
        # 0x0D: "GET_S_STATUS",
        # 0x0E: "BUILD_CHKSUM",
        # 0x10: "CLEAR_MEMORY",
        # 0x18: "PROGRAM",
        # 0x22: "PROGRAM_6",
        # 0x19: "MOVE",
        # 0x05: "TEST",
        # 0x09: "GET_ACTIVE_CAL_PAGE",
        # 0x08: "START_STOP_ALL",
        # 0x20: "DIAG_SERVICE",
        # 0x21: "ACTION_SERVICE"

ccp_command =  CCPCommand()
ccp_command.model_dump()

{'connect': 1,
 'set_mta': 2,
 'disconnect': 7,
 'download': 3,
 'download6': 35,
 'upload': 4,
 'short_upload': 15,
 'get_seed': 18,
 'get_ccp_version': 27}

In [ ]:
xcp_calib_from_xcpjson = Get_XCPCalib_From_XCPJSon(args.input)
xcp_calib_from_xcpjson

XCPCalib(config=XCPConfig(channel=3, download_can_id=630, upload_can_id=631), data=[{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000025...344',
    'value_array_view': array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
         4436.   ],
       [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
         4436.   ],
       [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
         4436.   ],
       ...,
       [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
         2034.368],
       [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
         1859.521],
       [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
         1690.249]], dtype=float32),
    'value_bytes': "b'\\x00'...b'D'",
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}])

In [ ]:

xcp_data = Generate_Init_XCPData_From_A2L(
    a2l=args.a2l, keys=args.leaves, node_path=args.node_path
)

#  address from xcp data file should align with the address from xcp calib file
test_eq(xcp_data.address, xcp_calib_from_xcpjson.data[0].address)

# validate the model
try:
    XCPData.model_validate(xcp_data)
except ValidationError as exc:
    print(exc)

{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000000...000',
    'value_array_view': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
    'value_bytes': "b'\\x00'...b'\\x00'",
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}

In [ ]:
# type(args.channel), type(args.download_id), args.upload_id, args.download, args.diff_flashing

In [ ]:
xcp_data.value = xcp_calib_from_xcpjson.data[0].value
pprint(xcp_data)
xcp_data.value_array_view[0,2], xcp_data.value_array_view[2,0]

{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000025...344',
    'value_array_view': array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
         4436.   ],
       [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
         4436.   ],
       [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
         4436.   ],
       ...,
       [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
         2034.368],
       [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
         1859.521],
       [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
         1690.249]], dtype=float32),
    'value_bytes': "b'\\x00'...b'D'",
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}


(1135.317, -1338.5337)

In [ ]:

xcp_calib = XCPCalib(
    config=XCPConfig(
        channel=args.channel_serial_number, download=args.download_can_id, upload=args.upload_can_id
    ),
    data=[xcp_data],
)

In [ ]:
pprint(xcp_calib)

XCPCalib(config=XCPConfig(channel=3, download_can_id=630, upload_can_id=631), data=[{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000025...344',
    'value_array_view': array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
         4436.   ],
       [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
         4436.   ],
       [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
         4436.   ],
       ...,
       [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
         2034.368],
       [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
         1859.521],
       [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
         1690.249]], dtype=float32),
    'value_bytes': "b'\\x00'...b'D'",
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}])


In [ ]:
npa =  xcp_calib.data[0].value_array_view
npa.shape, npa.dtype, npa
len(xcp_calib.data[0].value_bytes), xcp_calib.data[0].value_bytes

len(npa.tobytes()), npa.tobytes()
test_eq(npa.tobytes(), xcp_calib.data[0].value_bytes)
xcp_calib.data[0].value

((14, 17),
 dtype('float32'),
 array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
          4436.   ],
        [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
          4436.   ],
        [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
          4436.   ],
        ...,
        [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
          2034.368],
        [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
          1859.521],
        [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
          1690.249]], dtype=float32))

(952,
 b'\x00\x00\x00\x00%\xea\x8dD%\xea\x8dD%\xea\x8dD%\xea\x8dD\xdb4\x95D\xb3\x1e\xb6D\x8c\x08\xd7Dd\xf2\xf7D\x1en\x0cE\x81\x1d%E\xe3\xcc=EF|VE\x85\xed\x83E\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\x00\x00\x00\x00\xed\x16\x14C\xed\x16\x14CN\xba\x8dC\x1ay\x1eD\xcbL`D>\x10\x91D\x17\xfa\xb1D\xef\xe3\xd2D\xc8\xcd\xf3DF\x96\x12E\xa9E+E\x0b\xf5CE\xd0SuE\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\x14Q\xa7\xc4\x0bVP\xc4\xdd\x13\xa4\xc3\xb8\x081CM\xa9\xf9CV%AD\x84|\x81D\\f\xa2D5P\xc3D\r:\xe4Di\xcc\nE\xcb{#E.+<E\xf3\x89mE\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\xff\xfc\x0f\xc5\xa4\xfc\xd0\xc4K\xff\x81\xc4\xc8\x07\xcc\xc39\xdb_Cv\xa0\tD\xf8\xd7GD\xbd\x07\x83D\x7f#\xa2D@?\xc1D\xe2\xe8\xefDBI\x0fE\x13\x9e&E\xb5GUE\xab\xf8\x81E\x00\xa0\x8aE\x00\xa0\x8aEw\x1f\r\xc5\x9fj\xda\xc4Q\x96\x9a\xc4\x07\x845\xc4\xabmW\xc3b\x9a\x93C\xeb\x8d\x18D?)SDJ\xe2\x86D\xf4/\xa4Dt$\xd0D\xf3\x18\xfcD\xb9\x06\x14E9\xfb?E\xb8\xefkE\x00\xa0\x8aE\x00\xa0\x8aEH>\t\xc5\x1d\xd0\xdc\xc4\xa9#\xa7\xc4l\xeeb\xc4\n+\xef\xc3\xdc\xc9C\xc2\x

(952,
 b'\x00\x00\x00\x00%\xea\x8dD%\xea\x8dD%\xea\x8dD%\xea\x8dD\xdb4\x95D\xb3\x1e\xb6D\x8c\x08\xd7Dd\xf2\xf7D\x1en\x0cE\x81\x1d%E\xe3\xcc=EF|VE\x85\xed\x83E\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\x00\x00\x00\x00\xed\x16\x14C\xed\x16\x14CN\xba\x8dC\x1ay\x1eD\xcbL`D>\x10\x91D\x17\xfa\xb1D\xef\xe3\xd2D\xc8\xcd\xf3DF\x96\x12E\xa9E+E\x0b\xf5CE\xd0SuE\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\x14Q\xa7\xc4\x0bVP\xc4\xdd\x13\xa4\xc3\xb8\x081CM\xa9\xf9CV%AD\x84|\x81D\\f\xa2D5P\xc3D\r:\xe4Di\xcc\nE\xcb{#E.+<E\xf3\x89mE\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\xff\xfc\x0f\xc5\xa4\xfc\xd0\xc4K\xff\x81\xc4\xc8\x07\xcc\xc39\xdb_Cv\xa0\tD\xf8\xd7GD\xbd\x07\x83D\x7f#\xa2D@?\xc1D\xe2\xe8\xefDBI\x0fE\x13\x9e&E\xb5GUE\xab\xf8\x81E\x00\xa0\x8aE\x00\xa0\x8aEw\x1f\r\xc5\x9fj\xda\xc4Q\x96\x9a\xc4\x07\x845\xc4\xabmW\xc3b\x9a\x93C\xeb\x8d\x18D?)SDJ\xe2\x86D\xf4/\xa4Dt$\xd0D\xf3\x18\xfcD\xb9\x06\x14E9\xfb?E\xb8\xefkE\x00\xa0\x8aE\x00\xa0\x8aEH>\t\xc5\x1d\xd0\xdc\xc4\xa9#\xa7\xc4l\xeeb\xc4\n+\xef\xc3\xdc\xc9C\xc2\x

'0000000025ea8d4425ea8d4425ea8d4425ea8d44db349544b31eb6448c08d74464f2f7441e6e0c45811d2545e3cc3d45467c564585ed834500a08a4500a08a4500a08a4500000000ed161443ed1614434eba8d431a791e44cb4c60443e10914417fab144efe3d244c8cdf34446961245a9452b450bf54345d053754500a08a4500a08a4500a08a451451a7c40b5650c4dd13a4c3b80831434da9f94356254144847c81445c66a2443550c3440d3ae44469cc0a45cb7b23452e2b3c45f3896d4500a08a4500a08a4500a08a45fffc0fc5a4fcd0c44bff81c4c807ccc339db5f4376a00944f8d74744bd0783447f23a244403fc144e2e8ef4442490f45139e2645b5475545abf8814500a08a4500a08a45771f0dc59f6adac451969ac4078435c4ab6d57c3629a9343eb8d18443f2953444ae28644f42fa4447424d044f318fc44b906144539fb3f45b8ef6b4500a08a4500a08a45483e09c51dd0dcc4a923a7c46cee62c40a2befc3dcc943c29338be43be142944e413604485898b44e1c8b4443e08de44cda303452ae32c4586225645e3617f4500a08a45735904c5aa24dac46c96abc45f107ac4e5f31cc4ac5d7fc37928ea4212c3f443ff0b3e44f76e7144b5419f44efcbc5442856ec444eb51c45873f4345c1c96945f1378345efe1fcc4bac2d3c485a3aac44f8481c434ca30c49117bdc

In [ ]:
addr = bytes('7000aa2a', 'utf-8')
a = 0x7000aa2a
a

1879091754

In [ ]:
npb = npa[::-1]
# npb
buffer = [struct.pack("<f", x) for x in np.nditer(npa)]
# buffer
len(buffer)

238

### investigate int type and type size

In [ ]:
xcp_data = xcp_calib.data[0]
add = int(xcp_data.address, base=16)
xcp_data.address, add
hex(add), type(add), sys.getsizeof(add)
struct.calcsize('h'), struct.calcsize('i'), struct.calcsize('l'), struct.calcsize('L')
type(xcp_data.address),len(xcp_data.address)

('7000aa2a', 1879091754)

('0x7000aa2a', int, 32)

(2, 4, 8, 8)

(str, 8)

In [ ]:
#| export
def npa_to_packed_buffer(a: np.ndarray) -> str:
    """ convert a numpy array to a packed string buffer for flashing
    TODO: implementation as numpy ufunc

    Args:
        a (np.ndarray): input numpy array for flashing

    Returns:
        str: packed string buffer for flashing
    """
    b = [struct.pack("<f", x).hex() for x in np.nditer(a)]
    return ''.join(b)

In [ ]:

# buffer = [struct.pack("<f", x).hex() for x in np.nditer(npa)]
# buffer[::-1]
# len(buffer)
# buffer
# data = ''.join(buffer)
data = npa_to_packed_buffer(npa)
test_eq(data, xcp_calib.data[0].value)
# data

## convert a numpy array to a continuous hex string

In [ ]:
# npa.astype(np.float32).tobytes().hex()
buffer = npa.astype(np.float32).tobytes().hex()  ## == npa_to_packed_buffer(npa)
buffer, len(buffer)
test_eq(buffer, xcp_calib.data[0].value)

('0000000025ea8d4425ea8d4425ea8d4425ea8d44db349544b31eb6448c08d74464f2f7441e6e0c45811d2545e3cc3d45467c564585ed834500a08a4500a08a4500a08a4500000000ed161443ed1614434eba8d431a791e44cb4c60443e10914417fab144efe3d244c8cdf34446961245a9452b450bf54345d053754500a08a4500a08a4500a08a451451a7c40b5650c4dd13a4c3b80831434da9f94356254144847c81445c66a2443550c3440d3ae44469cc0a45cb7b23452e2b3c45f3896d4500a08a4500a08a4500a08a45fffc0fc5a4fcd0c44bff81c4c807ccc339db5f4376a00944f8d74744bd0783447f23a244403fc144e2e8ef4442490f45139e2645b5475545abf8814500a08a4500a08a45771f0dc59f6adac451969ac4078435c4ab6d57c3629a9343eb8d18443f2953444ae28644f42fa4447424d044f318fc44b906144539fb3f45b8ef6b4500a08a4500a08a45483e09c51dd0dcc4a923a7c46cee62c40a2befc3dcc943c29338be43be142944e413604485898b44e1c8b4443e08de44cda303452ae32c4586225645e3617f4500a08a45735904c5aa24dac46c96abc45f107ac4e5f31cc4ac5d7fc37928ea4212c3f443ff0b3e44f76e7144b5419f44efcbc5442856ec444eb51c45873f4345c1c96945f1378345efe1fcc4bac2d3c485a3aac44f8481c434ca30c49117bd

In [ ]:
#| export
def flash_xcp(xcp_calib: XCPCalib, data: pd.DataFrame, diff_flashing: bool=False, download: bool=True):
    """Summary
    Flash XCP data to target

    Args:
        xcp_calib (XCPCalib): XCP calibration as template, contains all the meta information except for data
        xcp_data (pd.DataFrame): input XCP data to be flashed, replace the value in xcp_calib
        diff_flashing (bool): Use differential flashing
        download (bool): Download or upload
    
    """
    
    # convert dataframe to a hex string to be flashed and assigned to XCPCalib field data
    xcp_calib.data = data.astype(np.float32).tobytes().hex()

    if download:
        if diff_flashing:
            raise NotImplementedError("Differential flashing not implemented yet")
        else:
            pass
        

    

# Caution

set the python3 of the virtualenv with the CAP_NET_RAW capability!

```bash
sudo setcap 'CAP_NET_RAW+eip CAP_NET_ADMIN+eip' /dpt/.pyenv/versions/miniconda3-3.11-24.1.2-0/envs/can/bin/python3.11
```

install sshpass, and create .sshpasswd.gpg file in home root
```bash
cd ~
sudo apt-get install sshpass
echo 'password_in_verbatim" > .sshpasswd
gpg -c .sshpasswd
ls | grep 'sshpasswd.gpg'
```

In [ ]:
args.channel_serial_number
bus = 'can'

3

In [ ]:

bus+str(args.channel_serial_number)

'can3'

In [ ]:
# install vcan interface with encrypted password to sudo 
if args.bus_type == 'SOCKET':
    bus='can'
else:  # 'VIRTUAL"
    bus='vcan'
device = bus+str(args.channel_serial_number)

os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo modprobe {bus}")


SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for veos: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



0

In [ ]:
os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link add dev {device} type {bus}")


SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for veos: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 

RTNETLINK answers: File exists


512

In [ ]:
device, bus, args.bit_rate

('vcan3', 'vcan', 500000)

In [ ]:
os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set {device} up type {bus} bitrate {args.bit_rate}")
# os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set up {device}")
# sudo ip link set can3 up type can bitrate 500000


SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for veos: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



0

In [ ]:
os.system(f"ip link show {device}")

10: can3: <NOARP,UP,LOWER_UP,ECHO> mtu 16 qdisc pfifo_fast state UP mode DEFAULT group default qlen 10
    link/can 


0

In [ ]:
bus = can.interface.Bus(bustype='socketcan', channel=device, bitrate=args.bit_rate)
bus.__dict__

{'socket': <socket.socket fd=81, family=29, type=3, proto=1, laddr=('can3',)>,
 'channel': 'can3',
 'channel_info': "socketcan channel 'can3'",
 '_bcm_sockets': {},
 '_is_filtered': True,
 '_task_id': 0,
 '_task_id_guard': <unlocked _thread.lock object>,
 '_can_protocol': <CanProtocol.CAN_20: 1>,
 '_periodic_tasks': [],
 '_filters': None}

In [ ]:
# cntr = 0
# can_data = 0x12345678
# message_to_send = can.Message(arbitration_id=args.download_can_id, 
#                             data=can_data, 
#                             is_extended_id=False) 	
# message_to_send

In [ ]:
args.channel_serial_number

3

# CCP via python-can 

In [ ]:
channel = 'can' + str(xcp_calib.config.channel)
channel

'can3'

In [ ]:
args

Namespace(protocol='CCP', bus_type='SOCKET', download=True, diff_mode=True, diff_threshold=0.001, bit_rate=500000, time_out=1.0, station_address=0, download_can_id=630, upload_can_id=631, a2l='/home/veos/devel/candycan/res/VBU_AI.json', node_path='/PROJECT/MODULE[]', leaves=['TQD_trqTrqSetNormal_MAP_v', 'VBU_L045A_CWP_05_09T_AImode_CM_single', 'Lookup2D_FLOAT32_IEEE', 'Lookup2D_X_FLOAT32_IEEE', 'Scalar_FLOAT32_IEEE', 'TQD_vVehSpd', 'TQD_vSgndSpd_MAP_y', 'TQD_pctAccPedPosFlt', 'TQD_pctAccPdl_MAP_x'], channel_serial_number=3, input='/home/veos/devel/candycan/res/download.json', output='/home/veos/devel/candycan/res/output.json')

In [ ]:
can_filters = [{'can_id': xcp_calib.config.upload_can_id, 'can_mask': 0x7FF}]
can_specs = ScapyCANSpecs(can_type='NATIVE', 
                        bus_type=args.bus_type, 
                        channel_serial_number=args.channel_serial_number,
                        download_can_id=xcp_calib.config.download_can_id,
                        upload_can_id=xcp_calib.config.upload_can_id,
                        can_filters=can_filters,
                        bit_rate=args.bit_rate,
                        time_out=1.0,
                        station_address=args.station_address,
                        cntr=0,
                        receive_own_messages=True,
                        download_upload=True
                        )
can_specs
can_specs.model_dump()

ScapyCANSpecs(bus_type='SOCKET', channel_serial_number=3, download_can_id=630, upload_can_id=631, can_filters=[CANFilter(can_id=631, can_mask=2047)], bit_rate=500000, time_out=1.0, station_address=0, cntr=0, receive_own_messages=True, download_upload=True, diff_mode=False, diff_threshold=0.001, last_download_data=None, channel='can3')

{'bus_type': 'SOCKET',
 'channel_serial_number': 3,
 'download_can_id': 630,
 'upload_can_id': 631,
 'can_filters': [{'can_id': 631, 'can_mask': 2047}],
 'bit_rate': 500000,
 'time_out': 1.0,
 'station_address': 0,
 'cntr': 0,
 'receive_own_messages': True,
 'download_upload': True,
 'diff_mode': False,
 'diff_threshold': 0.001,
 'last_download_data': None,
 'channel': 'can3'}

## create sample data for testing

In [ ]:
xcp_calib.config.download_can_id
hex(xcp_calib.config.download_can_id)

630

'0x276'

In [ ]:
xcp_calib.data

[{   'address': '7000aa2a',
     'dim': [14, 17],
     'name': 'TQD_trqTrqSetNormal_MAP_v',
     'type_size': 4,
     'value': '0000000025...344',
     'value_array_view': array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
          4436.   ],
        [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
          4436.   ],
        [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
          4436.   ],
        ...,
        [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
          2034.368],
        [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
          1859.521],
        [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
          1690.249]], dtype=float32),
     'value_bytes': "b'\\x00'...b'D'",
     'value_length': 4,
     'value_type': 'FLOAT32_IEEE'}]

In [ ]:
xcp_data = xcp_calib.data[0]
xcp_data.address, xcp_data.type_size, xcp_data.dim, xcp_data.value_array_view, xcp_data.value_bytes
len(xcp_data.value_bytes)
xcp_data.value

('7000aa2a',
 4,
 [14, 17],
 array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
          4436.   ],
        [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
          4436.   ],
        [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
          4436.   ],
        ...,
        [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
          2034.368],
        [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
          1859.521],
        [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
          1690.249]], dtype=float32),
 b'\x00\x00\x00\x00%\xea\x8dD%\xea\x8dD%\xea\x8dD%\xea\x8dD\xdb4\x95D\xb3\x1e\xb6D\x8c\x08\xd7Dd\xf2\xf7D\x1en\x0cE\x81\x1d%E\xe3\xcc=EF|VE\x85\xed\x83E\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\x00\x00\x00\x00\xed\x16\x14C\xed\x16\x14CN\xba\x8dC\x1ay\x1eD\xcbL`D>\x10\x91D\x17\xfa\xb1D\xef\xe3\xd2D\xc8\xcd\xf3DF\x96\x12E\xa9E+E\x0b\xf5CE\xd0SuE\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\x14Q\xa7\xc4\x0bVP\

952

'0000000025ea8d4425ea8d4425ea8d4425ea8d44db349544b31eb6448c08d74464f2f7441e6e0c45811d2545e3cc3d45467c564585ed834500a08a4500a08a4500a08a4500000000ed161443ed1614434eba8d431a791e44cb4c60443e10914417fab144efe3d244c8cdf34446961245a9452b450bf54345d053754500a08a4500a08a4500a08a451451a7c40b5650c4dd13a4c3b80831434da9f94356254144847c81445c66a2443550c3440d3ae44469cc0a45cb7b23452e2b3c45f3896d4500a08a4500a08a4500a08a45fffc0fc5a4fcd0c44bff81c4c807ccc339db5f4376a00944f8d74744bd0783447f23a244403fc144e2e8ef4442490f45139e2645b5475545abf8814500a08a4500a08a45771f0dc59f6adac451969ac4078435c4ab6d57c3629a9343eb8d18443f2953444ae28644f42fa4447424d044f318fc44b906144539fb3f45b8ef6b4500a08a4500a08a45483e09c51dd0dcc4a923a7c46cee62c40a2befc3dcc943c29338be43be142944e413604485898b44e1c8b4443e08de44cda303452ae32c4586225645e3617f4500a08a45735904c5aa24dac46c96abc45f107ac4e5f31cc4ac5d7fc37928ea4212c3f443ff0b3e44f76e7144b5419f44efcbc5442856ec444eb51c45873f4345c1c96945f1378345efe1fcc4bac2d3c485a3aac44f8481c434ca30c49117bdc

## Test bytes hex encodeing and decoding

In [ ]:
len_in_bytes = xcp_data.type_size * xcp_data.dim[0] * xcp_data.dim[1]
print(f"len_in_bytes: {len_in_bytes} = type_size: {xcp_data.type_size} x dim: {xcp_data.dim}")

download_times = len_in_bytes // 6
last_download_size = len_in_bytes % 6
print(f"download_times: {download_times}, last_download_size: {last_download_size}")


len_in_bytes: 952 = type_size: 4 x dim: [14, 17]
download_times: 158, last_download_size: 4


In [ ]:
xcp_data.value_bytes
len(xcp_data.value_bytes)
tile0 = xcp_data.value_bytes[0:6]
len(tile0), tile0, tile0.hex(), type(tile0)
tile1 = xcp_data.value_bytes[6:12]
len(tile1), tile1, tile1.hex()


b'\x00\x00\x00\x00%\xea\x8dD%\xea\x8dD%\xea\x8dD%\xea\x8dD\xdb4\x95D\xb3\x1e\xb6D\x8c\x08\xd7Dd\xf2\xf7D\x1en\x0cE\x81\x1d%E\xe3\xcc=EF|VE\x85\xed\x83E\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\x00\x00\x00\x00\xed\x16\x14C\xed\x16\x14CN\xba\x8dC\x1ay\x1eD\xcbL`D>\x10\x91D\x17\xfa\xb1D\xef\xe3\xd2D\xc8\xcd\xf3DF\x96\x12E\xa9E+E\x0b\xf5CE\xd0SuE\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\x14Q\xa7\xc4\x0bVP\xc4\xdd\x13\xa4\xc3\xb8\x081CM\xa9\xf9CV%AD\x84|\x81D\\f\xa2D5P\xc3D\r:\xe4Di\xcc\nE\xcb{#E.+<E\xf3\x89mE\x00\xa0\x8aE\x00\xa0\x8aE\x00\xa0\x8aE\xff\xfc\x0f\xc5\xa4\xfc\xd0\xc4K\xff\x81\xc4\xc8\x07\xcc\xc39\xdb_Cv\xa0\tD\xf8\xd7GD\xbd\x07\x83D\x7f#\xa2D@?\xc1D\xe2\xe8\xefDBI\x0fE\x13\x9e&E\xb5GUE\xab\xf8\x81E\x00\xa0\x8aE\x00\xa0\x8aEw\x1f\r\xc5\x9fj\xda\xc4Q\x96\x9a\xc4\x07\x845\xc4\xabmW\xc3b\x9a\x93C\xeb\x8d\x18D?)SDJ\xe2\x86D\xf4/\xa4Dt$\xd0D\xf3\x18\xfcD\xb9\x06\x14E9\xfb?E\xb8\xefkE\x00\xa0\x8aE\x00\xa0\x8aEH>\t\xc5\x1d\xd0\xdc\xc4\xa9#\xa7\xc4l\xeeb\xc4\n+\xef\xc3\xdc\xc9C\xc2\x938\xbe

952

(6, b'\x00\x00\x00\x00%\xea', '0000000025ea', bytes)

(6, b'\x8dD%\xea\x8dD', '8d4425ea8d44')

In [ ]:
tile0 = xcp_data.value_bytes[0:4]
len(tile0), tile0, tile0.hex(), type(tile0), struct.unpack("<f", tile0)

(4, b'\x00\x00\x00\x00', '00000000', bytes, (0.0,))

In [ ]:
tile1 = xcp_data.value_bytes[4:8]
len(tile1), tile1, tile1.hex(), struct.unpack("<f", tile1)

(4, b'%\xea\x8dD', '25ea8d44', (1135.3170166015625,))

In [ ]:
tile2 = xcp_data.value_bytes[8:12]
len(tile2), tile2, tile2.hex(), struct.unpack("<f", tile2)

(4, b'%\xea\x8dD', '25ea8d44', (1135.3170166015625,))

In [ ]:
st = 2*17*4
tile3 = xcp_data.value_bytes[st:st+4]
len(tile1), tile3, tile3.hex(), struct.unpack("<f", tile3)
xcp_data.value_array_view[2,0]
test_eq(struct.unpack("<f", tile3), xcp_data.value_array_view[2,0])

(4, b'\x14Q\xa7\xc4', '1451a7c4', (-1338.53369140625,))

-1338.5337

In [ ]:
tile3, len(tile3)
tile3.hex(), len(tile3.hex())
tile3.hex().encode(), len(tile3.hex().encode()), 
tile3.hex().encode().hex(), len(tile3.hex().encode().hex())
try:
    tile3.hex().hex()
except Exception as exc:
    print(exc)
try:
    tile3.decode("utf-8"), len(tile3.decode())
except Exception as exc:
    print(exc)


(b'\x14Q\xa7\xc4', 4)

('1451a7c4', 8)

(b'1451a7c4', 8)

('3134353161376334', 16)

'str' object has no attribute 'hex'
'utf-8' codec can't decode byte 0xa7 in position 2: invalid start byte


In [ ]:
tile3.decode('utf-16')
tile3.decode('utf-32','backslashreplace')
t3  = tile3.decode('utf-8','backslashreplace')
t3, len(t3), type(t3)
t4  = tile3.decode('utf-8','ignore')
t4, len(t4), type(t4)

'儔쒧'

'\\x14\\x51\\xa7\\xc4'

('\x14Q\\xa7\\xc4', 10, str)

('\x14Q', 2, str)

In [ ]:
type(tile3)
list(tile3)

bytes

[20, 81, 167, 196]

In [ ]:
ba_uploaded = bytearray()
ba_uploaded += tile3
ba_uploaded
ba_uploaded += tile2
ba_uploaded += tile1
ba_uploaded, len(ba_uploaded)

bytearray(b'\x14Q\xa7\xc4')

(bytearray(b'\x14Q\xa7\xc4%\xea\x8dD%\xea\x8dD'), 12)

In [ ]:
ba_uploaded = bytearray()
len_in_bytes = xcp_data.type_size * xcp_data.dim[0] * xcp_data.dim[1]
download_times = len_in_bytes // 6
last_download_size = len_in_bytes % 6
for tile in range(download_times):
    ba_uploaded += xcp_data.value_bytes[tile*6:(tile+1)*6]
if last_download_size:
    ba_uploaded += xcp_data.value_bytes[download_times*6:download_times*6+last_download_size]

len(ba_uploaded), ba_uploaded.hex()
test_eq(ba_uploaded, xcp_data.value_bytes)
test_eq(ba_uploaded.hex(), xcp_data.value)
ba_uploaded.hex()

(952,
 '0000000025ea8d4425ea8d4425ea8d4425ea8d44db349544b31eb6448c08d74464f2f7441e6e0c45811d2545e3cc3d45467c564585ed834500a08a4500a08a4500a08a4500000000ed161443ed1614434eba8d431a791e44cb4c60443e10914417fab144efe3d244c8cdf34446961245a9452b450bf54345d053754500a08a4500a08a4500a08a451451a7c40b5650c4dd13a4c3b80831434da9f94356254144847c81445c66a2443550c3440d3ae44469cc0a45cb7b23452e2b3c45f3896d4500a08a4500a08a4500a08a45fffc0fc5a4fcd0c44bff81c4c807ccc339db5f4376a00944f8d74744bd0783447f23a244403fc144e2e8ef4442490f45139e2645b5475545abf8814500a08a4500a08a45771f0dc59f6adac451969ac4078435c4ab6d57c3629a9343eb8d18443f2953444ae28644f42fa4447424d044f318fc44b906144539fb3f45b8ef6b4500a08a4500a08a45483e09c51dd0dcc4a923a7c46cee62c40a2befc3dcc943c29338be43be142944e413604485898b44e1c8b4443e08de44cda303452ae32c4586225645e3617f4500a08a45735904c5aa24dac46c96abc45f107ac4e5f31cc4ac5d7fc37928ea4212c3f443ff0b3e44f76e7144b5419f44efcbc5442856ec444eb51c45873f4345c1c96945f1378345efe1fcc4bac2d3c485a3aac44f8481c434ca30c4

'0000000025ea8d4425ea8d4425ea8d4425ea8d44db349544b31eb6448c08d74464f2f7441e6e0c45811d2545e3cc3d45467c564585ed834500a08a4500a08a4500a08a4500000000ed161443ed1614434eba8d431a791e44cb4c60443e10914417fab144efe3d244c8cdf34446961245a9452b450bf54345d053754500a08a4500a08a4500a08a451451a7c40b5650c4dd13a4c3b80831434da9f94356254144847c81445c66a2443550c3440d3ae44469cc0a45cb7b23452e2b3c45f3896d4500a08a4500a08a4500a08a45fffc0fc5a4fcd0c44bff81c4c807ccc339db5f4376a00944f8d74744bd0783447f23a244403fc144e2e8ef4442490f45139e2645b5475545abf8814500a08a4500a08a45771f0dc59f6adac451969ac4078435c4ab6d57c3629a9343eb8d18443f2953444ae28644f42fa4447424d044f318fc44b906144539fb3f45b8ef6b4500a08a4500a08a45483e09c51dd0dcc4a923a7c46cee62c40a2befc3dcc943c29338be43be142944e413604485898b44e1c8b4443e08de44cda303452ae32c4586225645e3617f4500a08a45735904c5aa24dac46c96abc45f107ac4e5f31cc4ac5d7fc37928ea4212c3f443ff0b3e44f76e7144b5419f44efcbc5442856ec444eb51c45873f4345c1c96945f1378345efe1fcc4bac2d3c485a3aac44f8481c434ca30c49117bdc

## CRO for connection

### Create can data as bytes of little endianness

In [ ]:
b1 = ccp_command.connect.to_bytes(byteorder='little', length=1)
ccp_command.connect, b1, b1.hex()
cntr = 2
b2 = cntr.to_bytes(byteorder='little', length=1)
b2, b2.hex()
can_data =  b1 + b2
can_data, can_data.hex()


(1, b'\x01', '01')

(b'\x02', '02')

(b'\x01\x02', '0102')

In [ ]:
cntr, ccp_command.connect, can_specs.station_address

cro_little = struct.pack("@BB", ccp_command.connect, cntr) + can_specs.station_address.to_bytes(byteorder='little', length=2)
cro_big = can_specs.station_address.to_bytes(byteorder='big', length=2) + struct.pack(">BB", cntr, ccp_command.connect) 
cro_little, cro_big
cro = cro_big
cro, cro.hex()

struct.unpack("@BBH", cro)

(5, 1, 0)

(b'\x01\x05\x00\x00', b'\x00\x00\x05\x01')

(b'\x00\x00\x05\x01', '00000501')

(0, 0, 261)

In [ ]:
# cro_little = struct.pack("@bb", ccp_command.connect, cntr) + can_specs.station_address.to_bytes(byteorder='little', length=2)
# cro_big = can_specs.station_address.to_bytes(byteorder='big', length=2) + struct.pack(">bb", cntr, ccp_command.connect) 
cro_little = struct.pack("@bb", ccp_command.connect, cntr) 
cro_big = struct.pack(">bb", cntr, ccp_command.connect) 
cro = cro_big
cro, cro.hex()
# struct.unpack("@bbh", cro)
struct.unpack("@bb", cro)

(b'\x01\x02', '0102')

(1, 2)

In [ ]:
try:
    d = struct.pack("@cc", cntr, ccp_command.connect)
    d, d.hex()
except Exception as exc:
    print(exc)

char format requires a bytes object of length 1


In [ ]:
device, can_specs.bit_rate, can_specs.can_filters, can_specs.station_address
cro

('can3', 500000, [CANFilter(can_id=631, can_mask=2047)], 0)

b'\x01\x02'

In [ ]:
# send connect message
# with can.interface.Bus(bustype='socketcan', channel=device, bitrate=can_specs.bit_rate, filter=can_specs.can_filters) as bus:
# with can.interface.Bus(interface='socketcan', channel=device, bitrate=can_specs.bit_rate) as bus:
with can.interface.Bus(interface='socketcan', channel=device, bitrate=can_specs.bit_rate, receive_own_message=True) as bus:
    msg = can.Message(
        arbitration_id=can_specs.download_can_id, 
        data=cro, 
        is_extended_id=False)
    try:
        bus.send(msg)
        cntr += 1
        print(f"Message sent: {msg} on {bus.channel_info} cntr: {cntr-1}")
    except can.CanError:
        print("Message NOT sent")
    
    dto = bus.recv(timeout=can_specs.time_out)
    dto
    

Message sent: Timestamp:        0.000000    ID:      276    S Rx                DL:  2    01 02 on socketcan channel 'can3' cntr: 4


In [ ]:
can_specs.time_out

1.0

In [ ]:
# with can.interface.Bus(interface='socketcan', channel=device, bitrate=can_specs.bit_rate, filter=can_specs.can_filters):
with can.interface.Bus(interface='socketcan', channel=device, bitrate=can_specs.bit_rate) as bus:
    dto: can.Message = bus.recv(timeout=can_specs.time_out)
    print(dto)

None


In [ ]:
# while(True):
with can.interface.Bus(interface='socketcan', channel=device, bitrate=can_specs.bit_rate) as bus:
    for msg in bus:
        print(msg)
# print('Message received: ', dto.arbitration_id, dto.data, dto.timestamp, dto.dlc)


In [ ]:
# # send connect message
# with can.interface.Bus(bustype='socketcan', channel=device, bitrate=can_specs.bit_rate, filter=can_specs.can_filters) as bus:
#     bus.send(msg)
#     cntr += 1
#     print(f"Message sent: {msg}, cntr: {cntr-1}")
#     while(True):
#         bus = can.interface.Bus(bustype='socketcan', channel=device, bitrate=can_specs.bit_rate, filter=can_specs.can_filters)
#         dto: can.Message = bus.recv()
#         print(dto)
#         # print('Message received: ', dto.arbitration_id, dto.data, dto.timestamp, dto.dlc)
#         if dto.dlc != 3 or dto.arbitration_id != can_specs.upload_can_id:
#             continue
#         pid, err_code , cntr_ret = struct.unpack('@BBB', dto.data)
#         if pid != 0xff:
#             continue
#         if cntr_ret == cntr and err_code == 0:
#             print(f"Connected: {pid}, {err_code}, {cntr}")
#             break
#         else:
#             print(f"Error: {pid}, {err_code}, {cntr}")
#             break

### send SET_MTA message

In [ ]:
xcp_data.address
addr = int(xcp_data.address, 16)
addr, hex(addr)
a_bytes_little = struct.pack("@I", addr)
a_bytes_big = struct.pack(">I", addr)
a_bytes_little, a_bytes_big

'7000aa2a'

(1879091754, '0x7000aa2a')

(b'*\xaa\x00p', b'p\x00\xaa*')

In [ ]:
# create CAN message for ccp SET_MTA CRO
xcp_data.address, cntr, ccp_command.set_mta, hex(addr), struct.pack("@I", addr)
can_data_big = struct.pack(">IBBBB", int(xcp_data.address,16), 0x00, 0x00, cntr, ccp_command.set_mta)
can_data_little = struct.pack("@BBBBI", ccp_command.set_mta, cntr, 0x00, 0x00, int(xcp_data.address,16))
can_data_little, can_data_big
can_data = can_data_little
cro = can.Message(arbitration_id=args.download_can_id, 
                            data=can_data, 
                            is_extended_id=False)	
cro

('7000aa2a', 3, 2, '0x7000aa2a', b'*\xaa\x00p')

(b'\x02\x03\x00\x00*\xaa\x00p', b'p\x00\xaa*\x00\x00\x03\x02')

can.Message(timestamp=0.0, arbitration_id=0x276, is_extended_id=False, dlc=8, data=[0x2, 0x3, 0x0, 0x0, 0x2a, 0xaa, 0x0, 0x70])

In [ ]:
# # set MTA
# with can.interface.Bus(bustype='socketcan', channel=device, bitrate=args.bit_rate, filter=can_specs.can_filters) as bus:
#     bus.send(cro)
#     cntr += 1
#     print(f"Message sent: {cro}, cntr: {cntr-1}")
    
#     while(True):
#         dto: can.Message = bus.recv()
#         print('Message received: ', dto.arbitration_id, dto.data, dto.timestamp, dto.dlc)
        
#         if dto.dlc != 3 or dto.arbitration_id != can_specs.upload_can_id:
#             continue
#         pid, err_code , cntr_ret = struct.unpack('@BBB', dto.data)
#         if pid != 0xff:
#             continue
#         if cntr_ret == cntr and err_code == 0:
#             print(f"Connected: {pid}, {err_code}, {cntr}")
#             break
#         else:
#             print(f"Error: {pid}, {err_code}, {cntr}")
#             break

## Download from target

In [ ]:

# can_data_big = struct.pack(">BBBBBBBB", int(xcp_data.address,16), 0x00, 0x00, cntr, ccp_command.download6)
can_data_little = struct.pack("@BB", ccp_command.download6, cntr)
ccp_command.download6, cntr, can_data_little


(35, 3, b'#\x03')

In [ ]:

xcp_data.value_bytes[0:6]
b = xcp_data.value_bytes[0]
b, b.to_bytes(), can_data_little + b.to_bytes()
can_data_little + xcp_data.value_bytes[0:6]

b'\x00\x00\x00\x00%\xea'

(0, b'\x00', b'#\x03\x00')

b'#\x03\x00\x00\x00\x00%\xea'

In [ ]:
tile = 0
for i in range(6):
    b = xcp_data.value_bytes[tile+i]
    can_data_little += b.to_bytes()
    b, can_data_little
# xcp_data.value_bytes[i*6:(i+1)*6]
can_data_little

(0, b'#\x03\x00')

(0, b'#\x03\x00\x00')

(0, b'#\x03\x00\x00\x00')

(0, b'#\x03\x00\x00\x00\x00')

(37, b'#\x03\x00\x00\x00\x00%')

(234, b'#\x03\x00\x00\x00\x00%\xea')

b'#\x03\x00\x00\x00\x00%\xea'

### loop over XCPCalib data array

In [ ]:
ccp_command

CCPCommand(connect=1, set_mta=2, disconnect=7, download=3, download6=35, upload=4, short_upload=15, get_seed=18, get_ccp_version=27)

In [ ]:

# for i in range(download_times):
#     # Create CAN message for ccp DOWNLOAD CRO
#     # xcp_data.address, cntr, ccp_command.set_mta, hex(addr), struct.pack("@I", addr)
#     # can_data_big = xcp_data.value_bytes[i*6:(i+1)*6:-1] + can_data_big
#     can_data_little = struct.pack("@BB", ccp_command.download6, cntr)
#     can_data_little += xcp_data.value_bytes[i*6:(i+1)*6]
#     # can_data_little, can_data_big
#     can_data = can_data_little
#     cro = can.Message(arbitration_id=args.download_can_id, data=can_data, is_extended_id=False)	
    
#     # Download data 
#     with can.interface.Bus(bustype='socketcan', channel=device, bitrate=args.bit_rate, filter=can_specs.can_filters) as bus:
#         bus.send(cro)
#         cntr += 1
#         print(f"Message sent: {cro}, cntr: {cntr-1}")

#         while(True):
#             dto: can.Message = bus.recv()
#             print('Message received: ', dto.arbitration_id, dto.data, dto.timestamp, dto.dlc)

#             # if dto.arbitration_id != can_specs.upload_can_id:
#             #     continue
#             pid, err_code , cntr_ret, mta0_ext, mta0_addr = struct.unpack('@BBBBI', dto.data)
#             if pid != 0xff:
#                 continue
#             if cntr_ret == cntr and err_code == 0:
#                 print(f"Downloaded: mta0_ext({mta0_ext}), mta0_add({mta0_addr}), {cntr}")
#                 break
#             else:
#                 print(f"Error: {pid}, {err_code}, {cntr}")
#                 break
#     # cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DNLOAD_6(data=d.value_bytes[i*6:(i+1)*6])
#     if i%100==0:
#         print(f"i: {i}, cntr: {cntr} cro: {cro}")

### Download the last tile

In [ ]:
# i = download_times * 6
# # can_data_big = xcp_data.value_bytes[i*6:(i+1)*6:-1] + can_data_big
# can_data_little = struct.pack("@BB", ccp_command.download, cntr)
# can_data_little += xcp_data.value_bytes[i:i+last_download_size]
# # can_data_little, can_data_big
# can_data = can_data_little
# cro = can.Message(arbitration_id=args.download_can_id, data=can_data, is_extended_id=False)	
    
# # Download data 
# with can.interface.Bus(bustype='socketcan', channel=device, bitrate=args.bit_rate, filter=can_specs.can_filters) as bus:
#     bus.send(cro)
#     cntr += 1
#     print(f"Message sent: {cro}, cntr: {cntr-1}")

#     while(True):
#         dto: can.Message = bus.recv()
#         print('Message received: ', dto.arbitration_id, dto.data, dto.timestamp, dto.dlc)

#         # if dto.arbitration_id != can_specs.upload_can_id:
#         #     continue
#         pid, err_code , cntr_ret, mta0_ext, mta0_addr = struct.unpack('@BBBBI', dto.data)
#         if pid != 0xff:
#             continue
#         if cntr_ret == cntr and err_code == 0:
#             print(f"Downloaded: mta0_ext({mta0_ext}), mta0_add({mta0_addr}), {cntr}")
#             break
#         else:
#             print(f"Error: {pid}, {err_code}, {cntr}")
#             break

## Disconnect target ecu

In [ ]:
# cro = struct.pack("@bbb", ccp_command.disconnect, cntr, 0x01) + b'\x00' + can_specs.station_address.to_bytes(byteorder='little', length=2) 
# cro, cro.hex()
# struct.unpack("@bbbbh", cro)

In [ ]:
# # send disconnect message
# with can.interface.Bus(bustype='socketcan', channel=device, bitrate=can_specs.bit_rate, filter=can_specs.can_filters) as bus:
#     bus.send(cro)
#     cntr += 1
#     print(f"Message sent: {cro}, cntr: {cntr-1}")
    
#     while(True):
#         dto: can.Message = bus.recv()
#         print('Message received: ', dto.arbitration_id, dto.data, dto.timestamp, dto.dlc)
        
#         if dto.dlc != 3 or dto.arbitration_id != can_specs.upload_can_id:
#             continue
#         pid, err_code , cntr_ret = struct.unpack('@bbb', dto.data)
#         if pid != 0xff:
#             continue
#         if cntr_ret == cntr and err_code == 0:
#             print(f"Connected: {pid}, {err_code}, {cntr}")
#             break
#         else:
#             print(f"Error: {pid}, {err_code}, {cntr}")
#             break

## Three context managers for CCP

In [ ]:
#| export
@contextlib.contextmanager
def can_context(can_specs: ScapyCANSpecs):
    """Summary
    Context manager for scapy CAN socket

    Args:
        can_specs (ScapyCANSpecs): CAN specs including can type, bus type, channel, etc.

    Yields:
        Bus: Python-CAN Bus object
    """
    # create a socket
 

In [ ]:
#| export
@contextlib.contextmanager
def SET_MTA_context(can_specs: ScapyCANSpecs, bus: can.interface.Bus, data: XCPData):
    """Summary
    Context manager for scapy set_mta 

    Args:
        channel (str): CAN channel to use, default is vcan0

    Yields:
        CAN: packdet for CAN message 
    """

    # SET_MTA 
    # cro = CCP(identifier=can_specs.download_can_id)/CRO(ctr=can_specs.cntr)/SET_MTA(address=int(data.address, 16))
    # dto = sock.sr1(cro, timeout=can_specs.time_out)
    # assert dto.return_code == 0x00
    # can_specs.cntr += 1
    # try:
    #     yield dto
    # except TimeoutError:
    #     raise TimeoutError(f"CAN socket timeout: {can_specs.time_out} seconds")
    # except Exception as e:
    #     raise e
    # finally:
    #     pass  # do nothing, just pray it'll be OK. Crapy CCP!
    


In [ ]:
#| export
@contextlib.contextmanager
def XLOAD_context(can_specs: ScapyCANSpecs, bus: can.interface.Bus, data: XCPData, start_index: int, tile_size: int):
    """Summary
    Context manager for scapy load (download or upload)

    Args:
        channel (str): CAN channel to use, default is vcan0

    Yields:
        CANSocket: CAN socket object
    """
    pass

## Downloading and uploading with context managers

In [ ]:
#| export
def upload_calib_data2(xcp_calib: XCPCalib, 
                        can_specs: ScapyCANSpecs
                        )->None:

    """Summary
    Upload XCP calibration data from target to host, the result will update the xcp_calib.data field

    Args:
        xcp_calib (XCPCalib): XCP calibration  to be uploaded from the target to host
        diff_flashing (bool): Use differential flashing
    """

    # init counter
    cntr = 0
    pass

In [ ]:
#| export
def downlod_calib_data2(xcp_calib: XCPCalib, 
                        can_specs: ScapyCANSpecs
                        )->None:
    """Summary
    Download XCP calibration data to target use scapy_can_context

    Args:
        xcp_calib (XCPCalib): XCP calibration  to be downloaded into the target
    """

    assert can_specs.download_upload==True, f"Check can_specs.download_upload flag, it should be True for download"
    if can_specs.diff_mode :
        if can_specs.last_download_data is None:  # diff mode in the first run, needs to upload first to populate last_download_data
            last_xcp_calib = XCPCalib(config=xcp_calib.config, data=xcp_calib.data)
            can_specs.download_upload = False
            upload_calib_data2(xcp_calib=last_xcp_calib, can_specs=can_specs)
            can_specs.last_download_data = last_xcp_calib.data
            can_specs.download_upload = True
        # calculate the difference between the last downloaded data and the current data
        assert len(can_specs.last_download_data)==len(xcp_calib.data), "XCPData list length is not the same"
        data_pair = zip(can_specs.last_download_data, xcp_calib.data)
        xcp_data = []
        for d0, d1 in zip(xcp_calib.data, can_specs.last_download_data):
            assert d0.is_compatible(d1), f"incompatible data {d0} vs {d1}"
            diff_array_index_2d = np.where((d0.value_array_view - d1.value_array_view) > can_specs.diff_threshold)
            diff_array_index_1d = np.ravel_multi_index(diff_array_index_2d, d0.dim, order='C')
            diff_array_value = d0[diff_array_index_2d]
            diff_array_address = d0.address_int + diff_array_index_1d * d0.type_size
            
            xcp_data += [XCPData(address=hex(address), 
                                value=value, 
                                name=d0.name, 
                                dim=d0.dim, 
                                value_type=d0.value_type, 
                                value_length=d0.value_length
                                ) 
                            for address, value in zip(diff_array_address, diff_array_value)
                        ]
    else:  # non-diff mode 
        xcp_data = xcp_calib.data
        
    try:
        pass
        # with can_context(can_specs=can_specs) as sock:
        #     for d in xcp_data:
        #         # SET_MTA
        #         with SET_MTA_context(can_specs=can_specs, sock=sock, data=d) as dto:
        #             assert dto.return_code==0x00, f"SET_MTA failed for {d.name} at {d.address}"
        #             # Determine message tiling
        #             len_in_bytes = d.type_size * d.dim[0] * d.dim[1]
        #             assert len_in_bytes == len(d.value_bytes)
        #             tile_size = 6  # 6 bytes per tile as defined in CCP for DNLOAD_6
        #             tiles = len_in_bytes //tile_size 
        #             last_tile = len_in_bytes % tile_size
        #             # Download full size tiles with DNLOAD_6
        #             for i in range(tiles):
        #                 start_index = i*tile_size
        #                 with XLOAD_context(can_specs=can_specs, sock=sock, data=d, start_index=start_index, tile_size=tile_size) as dto:
        #                     assert dto.return_code == 0x00, f"DNLOAD_6 failed at tile: {i}"

        #             start_index = tiles * tile_size
        #             with XLOAD_context(can_specs=can_specs, sock=sock, data=d, start_index=start_index, tile_size=last_tile) as dto:
        #                 assert dto.return_code == 0x00, f"DNLOAD failed at last tile: {i} of size {last_tile}"
    except Exception as e:
        print(e)
    
    # keep the last downloaded data for diff mode
    can_specs.last_download_data = xcp_calib.data

In [ ]:
can_filters = [{'can_id': xcp_calib.config.upload_can_id, 'can_mask': 0x7FF}]
can_specs = ScapyCANSpecs(can_type='NATIVE', 
                        bus_type='SOCKET', 
                        channel_serial_number=3,
                        download_can_id=xcp_calib.config.download_can_id,
                        upload_can_id=xcp_calib.config.upload_can_id,
                        can_filters=can_filters,
                        bit_rate=500_000,
                        time_out=1.0,
                        station_address=0x00,
                        cntr=0,
                        receive_own_messages=True,
                        download_upload=True
                        )
can_specs
can_specs.model_dump()

ScapyCANSpecs(bus_type='SOCKET', channel_serial_number=3, download_can_id=630, upload_can_id=631, can_filters=[CANFilter(can_id=631, can_mask=2047)], bit_rate=500000, time_out=1.0, station_address=0, cntr=0, receive_own_messages=True, download_upload=True, diff_mode=False, diff_threshold=0.001, last_download_data=None, channel='can3')

{'bus_type': 'SOCKET',
 'channel_serial_number': 3,
 'download_can_id': 630,
 'upload_can_id': 631,
 'can_filters': [{'can_id': 631, 'can_mask': 2047}],
 'bit_rate': 500000,
 'time_out': 1.0,
 'station_address': 0,
 'cntr': 0,
 'receive_own_messages': True,
 'download_upload': True,
 'diff_mode': False,
 'diff_threshold': 0.001,
 'last_download_data': None,
 'channel': 'can3'}

In [ ]:
xcp_calib.config.channel = 0
xcp_calib

XCPCalib(config=XCPConfig(channel=0, download_can_id=630, upload_can_id=631), data=[{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000025...344',
    'value_array_view': array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
         4436.   ],
       [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
         4436.   ],
       [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
         4436.   ],
       ...,
       [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
         2034.368],
       [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
         1859.521],
       [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
         1690.249]], dtype=float32),
    'value_bytes': "b'\\x00'...b'D'",
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}])

In [ ]:
# TODO cannot run test on CCP without a real or emulated ECU 
# downlod_calib_data2(xcp_calib, can_type='NATIVE', bus_type='VIRTUAL', bit_rate=500_000, timeout=1.0, station_address=0x00, diff_flashing=False)

## Test downloading and uploading

In [ ]:

xcp_calib_from_xcpjson = Get_XCPCalib_From_XCPJSon(args.input)

args.download_can_id = xcp_calib_from_xcpjson.config.download_can_id
args.upload_can_id = xcp_calib_from_xcpjson.config.upload_can_id
args.channel_serial_number = xcp_calib_from_xcpjson.config.channel

xcp_data = Generate_Init_XCPData_From_A2L(
    a2l=args.a2l, keys=args.leaves, node_path=args.node_path
)  # initial xcp_data has value 0
try:
    XCPData.model_validate(xcp_data)
except ValidationError as exc:
    print(exc)

# emulate torque table input as numpy array
xcp_data_value_npa = xcp_calib_from_xcpjson.data[0].value_array_view
xcp_data.value = xcp_data_value_npa.astype(np.float32).tobytes().hex()
pprint(xcp_data)

xcp_calib = XCPCalib(
    config=XCPConfig(
        channel=args.channel_serial_number, download=args.download_can_id, upload=args.upload_can_id
    ),
    data=[xcp_data],
)
pprint(xcp_calib)

can_filters = [{'can_id': xcp_calib.config.upload_can_id, 'can_mask': 0x7FF}]
cntr = 0
can_specs = ScapyCANSpecs(bus_type=args.bus_type,
                        channel_serial_number=args.channel_serial_number,
                        download_can_id=xcp_calib.config.download_can_id,
                        upload_can_id=xcp_calib.config.upload_can_id,
                        can_filters=can_filters,
                        bit_rate=args.bit_rate,
                        time_out=args.time_out,
                        station_address=args.station_address,
                        cntr=cntr,
                        receive_own_messages=True,
                        download_upload=args.download,  # CCP Upload mode
                        diff_mode = args.diff_mode,
                        diff_threshold= args.diff_threshold
                        )


{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000000...000',
    'value_array_view': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
    'value_bytes': "b'\\x00'...b'\\x00'",
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}

{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000025...344',
    'value_array_view': array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
         4436.   ],
       [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
         4436.   ],
       [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
         4436.   ],
       ...,
       [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
         2034.368],
       [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
         1859.521],
       [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
         1690.249]], dtype=float32),
    'value_bytes': "b'\\x00'...b'D'",
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}
XCPCalib(config=XCPConfig(channel=3, download_can_id=630, upload_can_id=631), data=[{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type

In [ ]:
can_specs

ScapyCANSpecs(bus_type='VIRTUAL', channel_serial_number=3, download_can_id=630, upload_can_id=631, can_filters=[CANFilter(can_id=631, can_mask=2047)], bit_rate=500000, time_out=1.0, station_address=0, cntr=0, receive_own_messages=True, download_upload=True, diff_mode=True, diff_threshold=0.001, last_download_data=None, channel='vcan3')

In [ ]:

# test uploading
# can_specs.download_upload = False                        
# upload_calib_data2(xcp_calib=xcp_calib, can_specs=can_specs)


# Release CAN device

In [ ]:

# close and remove vcan0
# !sshpass -v -p  asdf sudo ip link delete vcan0 

os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set down {device}")

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for veos: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



0

In [ ]:

# delete vcan0
os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link delete can{args.channel_serial_number}")

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for veos: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 

Cannot find device "can3"


256

In [ ]:
#| export
if __name__ == "__main__" and "__file__" in globals():  # only run if this file is called directly

    protocol = inquirer.select(
        message="What's the protocol?",
        choices=[
            Choice(value="CCP", name="CCP"),
            Choice(value="XCP", name="XCP"),
        ],
        default="CCP",
    ).execute()

    can = inquirer.select(
        message="What's the type of CAN?",
        choices=[
            Choice(value="NATIVE", name="Native Linux SocketCAN"),
            Choice(value="PYTHON", name="Python CAN"),
        ],
        default="NATIVE",
    ).execute()

    if can == 'NATIVE':
        bus = inquirer.select(
            message="What's the type of bus?",
            choices=[
                Choice(value="SOCKET", name="Physical CAN"),
                Choice(value="VIRTUAL", name="Virtual CAN"),
            ],
            default="SOCKET",
        ).execute()
    else:  # can == 'PYTHON'
        bus = inquirer.select(
            message="What's the type of bus?",
            choices=[
                Choice(value="SOCKET", name="Physical SocketCAN"),
                Choice(value="VIRTUAL", name="Virtual SocketCAN"),
                Choice(value="KVASER", name="Kvaser CAN"),
            ],
            default="SOCKET",
        ).execute()

    download = inquirer.confirm(
        message="Downloading(host->target)?",
        confirm_letter="y",
        reject_letter="n",
        default=True,
    ).execute()

    differential_flashing = inquirer.confirm(
        message="Differential Flashing?",
        confirm_letter="y",
        reject_letter="n",
        default=True,
    ).execute()

    a2l_file_path = inquirer.text(
        message="a2l file path",
        validate=EmptyInputValidator(),
        default='/res/vbu_ai.json'
    ).execute()

    # node_path = inquirer.text(
    #     message="node path",
    #     validate=EmptyInputValidator(),
    # 	default=r"/PROJECT/MODULE[]",
    # ).execute()

    # leaves = inquirer.text(
    #     message="leaves",
    #     validate=EmptyInputValidator(),
    # 	default=r"TQD_trqTrqSetNormal_MAP_v, VBU_L045A_CWP_05_09T_AImode_CM_single, Lookup2D_FLOAT32_IEEE, Lookup2D_X_FLOAT32_IEEE, Scalar_FLOAT32_IEEE, TQD_vVehSpd, TQD_vSgndSpd_MAP_y, TQD_pctAccPedPosFlt, TQD_pctAccPdl_MAP_x"
    # ).execute()

    # can_channel = inquirer.number(
    #     message="CAN channel for flashing",
    #     min_allowed=0,
    #     max_allowed=32,
    #     validate=EmptyInputValidator(),
    #     default=3,
    # ).execute()

    input_file_path = inquirer.text(
        message="Input file path",
        validate=EmptyInputValidator(),
        default="/res/download.json",
    ).execute()

    output_file_path = inquirer.text(
        message="Output file path",
        validate=EmptyInputValidator(),
        default="/res/output.json",
    ).execute()

    args = get_argparser().parse_args()
    args.protocol = protocol
    args.can_type = can
    args.bus_type = bus
    args.download = download
    args.diff_mode = differential_flashing
    # args.a2l = a2l_file_path
    # args.node_path = node_path
    # args.leaves = leaves
    args.input = repo.working_dir+input_file_path
    args.output = repo.working_dir+output_file_path
    pprint(args)

    xcp_calib_from_xcpjson = Get_XCPCalib_From_XCPJSon(args.input)

    args.download_can_id = xcp_calib_from_xcpjson.config.download_can_id
    args.upload_can_id = xcp_calib_from_xcpjson.config.upload_can_id
    args.channel_serial_number = xcp_calib_from_xcpjson.config.channel

    xcp_data = Generate_Init_XCPData_From_A2L(
        a2l=args.a2l, keys=args.leaves, node_path=args.node_path
    )  # initial xcp_data has value 0
    try:
        XCPData.model_validate(xcp_data)
    except ValidationError as exc:
        print(exc)

    # emulate torque table input as numpy array
    xcp_data_value_npa = xcp_calib_from_xcpjson.data[0].value_array_view
    xcp_data.value = xcp_data_value_npa.astype(np.float32).tobytes().hex()
    pprint(xcp_data)

    xcp_calib = XCPCalib(
        config=XCPConfig(
            channel=args.channel_serial_number, download=args.download_can_id, upload=args.upload_can_id
        ),
        data=[xcp_data],
    )
    pprint(xcp_calib)

    can_filters = [{'can_id': xcp_calib.config.upload_can_id, 'can_mask': 0x7FF}]
    cntr = 0
    can_specs = ScapyCANSpecs(can_type=args.can_type,
                            bus_type=args.bus_type,
                            channel_serial_number=args.channel_serial_number,
                            download_can_id=xcp_calib.config.download_can_id,
                            upload_can_id=xcp_calib.config.upload_can_id,
                            can_filters=can_filters,
                            bit_rate=args.bit_rate,
                            time_out=args.time_out,
                            station_address=args.station_address,
                            cntr=cntr,
                            receive_own_messages=True,
                            download_upload=args.download,  # CCP Upload mode
                            diff_mode = args.diff_mode,
                            diff_threshold= args.diff_threshold
                            )
    can_specs.download_upload = False                        
    upload_calib_data2(xcp_calib=xcp_calib, can_specs=can_specs)
    can_specs.download_upload = True                       
    downlod_calib_data2(xcp_calib=xcp_calib,can_specs=can_specs)
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()